In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
from tkinter import messagebox
from ttkthemes import ThemedStyle
from PIL import Image, ImageTk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler
import talib as ta
import torch
import torch.nn as nn
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
import joblib
import datetime
import warnings
import ttkthemes
import subprocess
import os


torch.manual_seed(0)

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, 
                            hidden_dim, 
                            num_layers, 
                            bias=True, 
                            batch_first=True, 
                            dropout=0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out


def train_model(mode):
    stock_symbols = stock_entry.get().upper()
    stocks = stock_symbols.split(", ")

    param_grid = {
        'hidden_dim': [64, 128],
        'num_layers': [1, 2],
        'num_epochs': [100, 150]
    }
    easy_param_grid = {
        'hidden_dim': [128],
        'num_layers': [1],
        'num_epochs': [100]
    }
    
    all_data = {}
    forword_split = 6
    
    window = tk.Toplevel(root)
    
    window.title("Training Results" if language == "english" else "訓練結果")
    text_area = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=40, height=10)
    text_area.pack(expand=True, fill='both')
    
    text_area.insert(tk.END, f"Training started. Please do not interrupt the process.\n" if language == "english" else f"開始訓練，過程請勿中斷\n")
    text_area.update()
    
    for stock_symbol in stocks:
        data = yf.Ticker(stock_symbol.strip()).history(period='max')
        start_df = pd.DataFrame(data)
        try:
            start_df.drop('Dividends', axis=1, inplace=True)
        except Exception as e:
            messagebox.showwarning("Warning" if language == "english" else "警告", f"Stock symbol {stock_symbol} not found." if language == "english" else f"查無 {stock_symbol} 這支股票")
            window.destroy()
            return
        start_df.drop('Stock Splits', axis=1, inplace=True)
        
        #加入技術指鰾
        start_df['MACD'] = ta.MACD(start_df['Close'])[0]
        start_df['ADX'] = ta.ADX(start_df['High'],start_df['Low'],start_df['Close'])
        start_df['CCI'] = ta.CCI(start_df['High'],start_df['Low'],start_df['Close'])
        start_df['K'], start_df['D'] = ta.STOCH(start_df['High'],start_df['Low'],start_df['Close'])
        start_df['MFI'] = ta.MFI(start_df['High'],start_df['Low'],start_df['Close'],start_df['Volume'])
        start_df['WILLR'] = ta.WILLR(start_df['High'],start_df['Low'],start_df['Close'])
        start_df.dropna(inplace=True)
        
        features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MACD', 
                    'ADX', 'CCI', 'K', 'D', 'MFI', 'WILLR']
        
        currentDateTime = datetime.datetime.now()
        date = currentDateTime.date()
        year = date.strftime("%Y")
        year = int(year)
        month = date.strftime("%m")
        month = int(month)
        
        if mode==0 or mode==1:
            start_year = 2008.5
            end_year = 2014
            
            if month < 7:
                num_segments = 2*(year-end_year+1)-1
            else:
                num_segments = 2*(year-end_year+1)
        elif mode==2:
            now_date = year
            num_segments = 1
            if month < 7:
                now_date += 0.5
            else:
                now_date += 1
            
            end_year = now_date-0.5
            start_year = end_year-5.5
            
        
        check_exist = 0
        check_exist_easy = 0
        all_error = 0
        
        for segment in range(num_segments):
            start_year += 0.5
            end_year += 0.5
    
            if mode==0:
                if os.path.exists(f"Model/{start_year}~{end_year}_{stock_symbol}.gz"):
                    text_area.insert(tk.END, f"{start_year}~{end_year}_{stock_symbol} already exists.\n" if language == "english" else f"{start_year}~{end_year}_{stock_symbol} 已經存在\n")
                    text_area.update()
                    continue
            elif mode==1:
                if os.path.exists(f"Model/{start_year}~{end_year}_{stock_symbol}.gz") or os.path.exists(f"Model/easy_{start_year}~{end_year}_{stock_symbol}.gz"):
                    if os.path.exists(f"{start_year}~{end_year}_{stock_symbol}.gz"):
                        check_exist += 1
                    if os.path.exists(f"Model/easy_{start_year}~{end_year}_{stock_symbol}.gz"):
                        check_exist_easy += 1

                    if check_exist > 0 and check_exist_easy > 0:
                        all_error +=1
                        text_area.insert(tk.END, f"{start_year}~{end_year}_{stock_symbol} does not exist, but advanced training is required.\n" if language == "english" else f"{start_year}~{end_year}_{stock_symbol} 雖然不存在，但須進行進階訓練\n")
                        text_area.update()
                        continue
                    else:
                        text_area.insert(tk.END, f"{start_year}~{end_year}_{stock_symbol} already exists or a more advanced module exists.\n" if language == "english" else f"{start_year}~{end_year}_{stock_symbol} 已經存在或擁有更完善的模組\n")
                        text_area.update()
                        continue
                    
            elif mode==2:
                if os.path.exists(f"Model/{start_year}~{end_year}_{stock_symbol}.gz") or os.path.exists(f"Model/easy_{start_year}~{end_year}_{stock_symbol}.gz"):
                    text_area.insert(tk.END, f"{start_year}~{end_year}_{stock_symbol} module does not need to be updated.\n" if language == "english" else f"{start_year}~{end_year}_{stock_symbol} 模組無需更新\n")
                    text_area.update()
                    continue
                elif os.path.exists(f"Model/{start_year-0.5}~{end_year-0.5}_{stock_symbol}.gz"):
                    mode=0
                elif os.path.exists(f"Model/easy_{start_year-0.5}~{end_year-0.5}_{stock_symbol}.gz"):
                    mode=1
                else:
                    all_error +=1
                    text_area.insert(tk.END, f"{start_year}~{end_year}_{stock_symbol} module is either not available or damaged. Please retrain.\n" if language == "english" else f"{start_year}~{end_year}_{stock_symbol} 模組尚未訓練或損毀，請重新訓練\n")
                    text_area.update()
                    continue
            
            df = start_df[ (start_df.index.year >= np.floor(start_year)) & (start_df.index.year <= end_year) &
                           (start_df.index.year + start_df.index.month / 12 >= start_year) &
                           (start_df.index.year + start_df.index.month / 12 <= end_year)]
            #print(start_year,' ~ ',end_year)
            train_data = df[(df.index.year >= np.floor(start_year)) &
                            (df.index.year + df.index.month / 12 <= end_year - 0.5)]

        
            test_data = df[(df.index.year + df.index.month / 12 > end_year - 0.5) &
                           (df.index.year + df.index.month / 12 <= end_year)]
           
    
            x_train = train_data[features].values
            y_train = train_data['Close'].values.reshape(-1, 1)
            x_test = test_data[features].values
            y_test = test_data['Close'].values.reshape(-1, 1)
    
            scaler = StandardScaler()
            scaler_y = StandardScaler()
            scaler.fit(x_train)
            scaler_y.fit(y_train)
            x_train = scaler.transform(x_train)
            y_train = scaler_y.transform(y_train)
            x_test = scaler.transform(x_test)
            y_test = scaler_y.transform(y_test) 
    
    
            x_train_batch = []
            x_test_batch = []

            roll_steps = 4
    
            for index in range(roll_steps-1, len(x_train)):
                x_train_batch.append(x_train[index-roll_steps+1:index])
            x_train_batch = np.array(x_train_batch)
    
            y_train = y_train[roll_steps-1:]
    
            for index in range(roll_steps-1, len(x_test)):
                x_test_batch.append(x_test[index-roll_steps+1:index])
            x_test_batch = np.array(x_test_batch)
    
            y_test = y_test[roll_steps-1:]
    
    
            x_train_batch = torch.from_numpy(x_train_batch).type(torch.Tensor)
            x_test_batch = torch.from_numpy(x_test_batch).type(torch.Tensor)
            y_train = torch.from_numpy(y_train).type(torch.Tensor)
            y_test = torch.from_numpy(y_test).type(torch.Tensor)
    
    
    
            input_dim = len(features)
            output_dim = 1
    
            best_rmse = float('inf') 
            best_params = None
            
            if mode==0:
                for params in ParameterGrid(param_grid):
                    temp_model = LSTM(input_dim=input_dim, 
                                 hidden_dim=params['hidden_dim'], 
                                 output_dim=output_dim, 
                                 num_layers=params['num_layers'])
        
                    criterion = torch.nn.MSELoss(reduction='mean')
                    optimizer = torch.optim.Adam(temp_model.parameters(), lr=0.01)
        
                    hist = np.zeros(params['num_epochs'])
                    for t in range(params['num_epochs']):
                        y_train_pred = temp_model(x_train_batch)
                        loss = criterion(y_train_pred, y_train)
                        hist[t] = loss.item()
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
        
                    y_test_pred = temp_model(x_test_batch)
                    rmse = np.sqrt(mean_squared_error(y_test.detach().numpy(), y_test_pred.detach().numpy()))
                    #print("參數:", params, "  RMSE:" , rmse)
                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_params = params
            elif mode==1:
                for params in ParameterGrid(easy_param_grid):
                    temp_model = LSTM(input_dim=input_dim, 
                                 hidden_dim=params['hidden_dim'], 
                                 output_dim=output_dim, 
                                 num_layers=params['num_layers'])
        
                    criterion = torch.nn.MSELoss(reduction='mean')
                    optimizer = torch.optim.Adam(temp_model.parameters(), lr=0.01)
        
                    hist = np.zeros(params['num_epochs'])
                    for t in range(params['num_epochs']):
                        y_train_pred = temp_model(x_train_batch)
                        loss = criterion(y_train_pred, y_train)
                        hist[t] = loss.item()
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
        
                    y_test_pred = temp_model(x_test_batch)
                    rmse = np.sqrt(mean_squared_error(y_test.detach().numpy(), y_test_pred.detach().numpy()))
                    #print("參數:", params, "  RMSE:" , rmse)
                    if rmse < best_rmse:
                        best_rmse = rmse
                        best_params = params
            
            
            text_area.insert(tk.END, f"{start_year}~{end_year}_{stock_symbol} best_RMSE = {best_rmse:.2f}.\n" if language == "english" else f"{start_year}~{end_year}_{stock_symbol} 最佳RMSE = {best_rmse:.2f}.\n")
            text_area.update()
            #print("最佳RMSE:", best_rmse)
            #print("最佳超參數:", best_params)
    
            model = LSTM(input_dim=input_dim, 
                          hidden_dim=best_params['hidden_dim'], 
                          output_dim=output_dim, 
                          num_layers=best_params['num_layers'])
    
            criterion = torch.nn.MSELoss(reduction='mean')
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
            
            
            hist = np.zeros(best_params['num_epochs'])
            for t in range(best_params['num_epochs']):
                y_train_pred = model(x_train_batch)
                loss = criterion(y_train_pred, y_train)
                hist[t] = loss.item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            
            if mode==0:
                file_path = os.path.join("Model", f"{start_year}~{end_year}_{stock_symbol}.gz")
                if os.path.exists(f"Model/easy_{start_year}~{end_year}_{stock_symbol}.gz"):
                    os.remove(f"Model/easy_{start_year}~{end_year}_{stock_symbol}.gz")
                joblib.dump(value=model,filename=file_path) 
            elif mode==1:
                file_path = os.path.join("Model", f"easy_{start_year}~{end_year}_{stock_symbol}.gz")
                joblib.dump(value=model,filename=file_path)
            text_area.insert(tk.END, f"{start_year}~{end_year}_{stock_symbol} training successful.\n" if language == "english" else f"{start_year}~{end_year}_{stock_symbol} 訓練成功\n")
            text_area.update()

        if all_error == 0:
            text_area.insert(tk.END, f"{stock_symbol} training successful.\n" if language == "english" else f"{stock_symbol} 訓練成功\n")
            text_area.update()
        else:
            text_area.insert(tk.END, f"{all_error} errors occurred, please check.\n" if language == "english" else f"發生了{all_error}個錯誤，請檢查\n")
            text_area.update()

def change_app():
    try:
        subprocess.Popen(["./analysis_program.exe"])
    except FileNotFoundError:
        messagebox.showwarning("Warning" if language == "english" else "警告", f"發生錯誤，無法開啟預測APP。" if language == "chinese" else f"An error occurred and could not open the prediction APP.")

In [4]:
language = "chinese"

def close_app():
    root.destroy()

def toggle_language():
    global language
    if language == "chinese":
        language = "english"
        root.title("Stock Prediction Application")
    else:
        language = "chinese"
        root.title("股票預測應用程式")
    update_language()


def update_language():
    if language == "chinese":
        label.config(text="請輸入所有要訓練的股票代碼（以逗號分隔，例如：AAPL, GOOG, AMD, AMZN）: ")
        train_button.config(text="訓練股票(進階版)")
        easy_train_button.config(text="訓練股票(簡易版)")
        update_button.config(text="模組更新(最近一次)")
        print_stocks_button.config(text="顯示已訓練的股票")
        change_app_button.config(text="切換到分析APP")
        exit_button.config(text="退出")
        language_button.config(text="Switch to English version")
        
    else:
        label.config(text="Please enter all the stock codes you want to train (separated by commas, e.g., AAPL, GOOG, AMD, AMZN): ")
        train_button.config(text="Train Stocks (Advanced Mode)")
        easy_train_button.config(text="Train Stocks (Easy Mode)")
        update_button.config(text="Update Models (Last Time)")
        print_stocks_button.config(text="Show Trained Stocks")
        change_app_button.config(text="Switch to Analysis App")
        exit_button.config(text="Exit")
        language_button.config(text="切換到中文版")


def print_finish_stocks(folder_path):
    stock_symbols = set()
    easy_stock_symbols = set()
    for filename in os.listdir(folder_path):
        if filename.endswith(".gz"):
            parts = filename.split("_")
            if len(parts) == 2:
                stock_symbol = parts[1].split(".")[0]
                stock_symbols.add(stock_symbol)
            elif len(parts) == 3:
                stock_symbol = parts[2].split(".")[0]
                easy_stock_symbols.add(stock_symbol)
    
    window = tk.Toplevel(root)
    window.title("Stock List" if language == "english" else "股票列表")
    window.geometry("400x500")
    
    label_stock_symbols_title = ttk.Label(window, text="Advanced Stock Models:")
    label_stock_symbols_title.pack()
    
    ttk.Label(window, text="").pack()
    
    for stock_symbol in stock_symbols:
        label_stock_symbol = ttk.Label(window, text=stock_symbol)
        label_stock_symbol.pack()

    ttk.Label(window, text="").pack()
    ttk.Label(window, text="").pack()
    
    label_easy_stock_symbols_title = ttk.Label(window, text="Easy Stock Models:")
    label_easy_stock_symbols_title.pack()
    
    ttk.Label(window, text="").pack()
    
    for easy_stock_symbol in easy_stock_symbols:
        label_easy_stock_symbol = ttk.Label(window, text=easy_stock_symbol)
        label_easy_stock_symbol.pack()

root = tk.Tk()
root.title("股票訓練應用程式")

style = ttkthemes.ThemedStyle(root)
style.set_theme("ubuntu")

image = Image.open('background.jpg')
background_image = ImageTk.PhotoImage(image)
background_label = tk.Label(root, image=background_image)
background_label.place(relwidth=1, relheight=1)

root.resizable(False, False)


label = ttk.Label(root, text="請輸入所有要訓練的股票代碼（以逗號分隔，例如：AAPL, GOOG, AMD, AMZN）: ", foreground="black" , font=('Helvetica', 12))
label.pack(padx=10, pady=10)

stock_entry = ttk.Entry(root, width=30)
stock_entry.pack(padx=10, pady=10)


style.configure('Custom.TButton', foreground='black', font=('Helvetica', 12))

train_button = ttk.Button(root, text="訓練股票(進階版)", style='Custom.TButton',  command=lambda: train_model(0))
train_button.pack(padx=10, pady=10)


easy_train_button = ttk.Button(root, text="訓練股票(簡易版)", style='Custom.TButton',  command=lambda: train_model(1))
easy_train_button.pack(padx=10, pady=10)

update_button = ttk.Button(root, text="模組更新(最近一次)", style='Custom.TButton',  command=lambda: train_model(2))
update_button.pack(padx=10, pady=10)

print_stocks_button = ttk.Button(root, text="顯示已訓練的股票", style='Custom.TButton', command=lambda: print_finish_stocks("Model/"))
print_stocks_button.pack(padx=10, pady=10)

change_app_button = ttk.Button(root, text="切換到分析APP", style='Custom.TButton',  command=change_app)
change_app_button.pack(padx=10, pady=10)

exit_button = ttk.Button(root, text="退出", style='Custom.TButton',  command=close_app)
exit_button.pack(padx=10, pady=10)

language_button = ttk.Button(root, text="Switch to English version" if language == "chinese" else "切換到中文版", style='Custom.TButton', command=toggle_language)
language_button.pack(padx=10, pady=10)

root.mainloop()

Could not get exchangeTimezoneName for ticker '' reason: 'chart'
: No timezone found, symbol may be delisted
C:\Users\jason\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
C:\Users\jason\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
C:\Users\jason\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dr